# Quantum Phase Estimation (QPE) Algorithm

The Quantum Phase Estimation (QPE) algorithm is a fundamental quantum algorithm used to estimate the phase (or eigenvalue) associated with a unitary operator. It plays a crucial role in quantum computing, especially in algorithms like Shor's algorithm for factoring.

In this notebook, we will:
1. Implement the QPE algorithm
2. Break down each step of the algorithm
3. Test QPE on a simple example


In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np


## Step 1: Import Necessary Libraries
- Qiskit:
  - QuantumCircuit for constructing quantum circuits
  - transpile to optimize circuits for the simulator
- AerSimulator as the backend for running and testing the quantum circuits
- NumPy for mathematical operations like pi


In [ ]:
def apply_qpe(unitary, n_qubits, state):
    qc = QuantumCircuit(n_qubits, len(state) + n_qubits)

    
    for qubit, bit in enumerate(reversed(state)):
        if bit == '1':
            qc.x(qubit)
    
    for qubit in range(n_qubits):
        qc.h(qubit)
    

    for i in range(n_qubits):
        for j in range(i + 1, n_qubits):
            qc.append(unitary, [j, i])
    
    qc.measure(range(n_qubits), range(n_qubits, n_qubits + n_qubits))
    return qc


## Step 2: QPE Circuit Construction
1. **State Initialization:**
   - Prepare the quantum state to which we want to apply phase estimation (e.g., a superposition state).
   - The state qubits are initialized based on the input state (in binary form).

2. **Hadamard Gates:**
   - Apply Hadamard gates to the ancilla qubits (the first n_qubits qubits) to create superposition.

3. **Controlled Unitary Gates:**
   - Apply the controlled version of the unitary operator to the qubits. The unitary operator is typically an operation whose eigenvalue we wish to estimate.
   - These gates apply the operator to the state qubits, with the ancilla qubits controlling how the operator acts.

4. **Measurement:**
   - Measure the ancilla qubits to extract the phase information from the quantum state.
   - The output of these measurements will give us an approximation of the phase of the unitary operator.


In [3]:
def run_qpe(input_state, unitary):
    n_qubits = len(input_state)
    qc = apply_qpe(unitary, n_qubits, input_state)
    
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit)
    result = job.result()
    counts = result.get_counts()
    return counts


## Step 3: Run QPE
1. **Input State:**
   - Prepare the input quantum state (in binary form).
   
2. **Apply QPE Circuit:**
   - Use the apply_qpe function to construct the quantum circuit based on the input state and the unitary operator.

3. **Simulate the Circuit:**
   - Simulate the quantum circuit using the AerSimulator.
   - Measure the results to estimate the phase.

4. **Return the Results:**
   - The measurement result (a binary string) represents an approximation of the phase. This is based on the outcome of the ancilla qubits after the controlled-unitary operations.


In [ ]:

def phase_shift_unitary(theta):
    qc = QuantumCircuit(2)
    qc.p(theta, 1)  
    return qc.to_gate()

# Test the QPE function
input_state = "0"
unitary = phase_shift_unitary(np.pi / 4)  
qpe_result = run_qpe(input_state, unitary)
print(qpe_result)


{'00': 514, '10': 510}


## Step 4: Test with Phase Shift Unit
1. **Phase Shift Unitary:**
   - A phase shift unitary operator $ U(\theta) $ is defined. It applies a phase shift on a qubit, changing its state.
   
2. **Test the QPE Function:**
   - The QPE is tested on a simple example where we apply a phase shift of $ \pi/4 $ to the quantum state.

3. **Output the Results:**
   - The result is printed, showing the measured phase in binary form. The closer the result is to the exact phase $ \pi/4 $, the more accurate the phase estimation.
